In [50]:
#import necessary packages
import numpy as np
import pandas as pd
import pickle

In [51]:
#importing team dictionaries using pickle
dict2020 = pickle.load(open("2020Dict.p","rb"))
dict2021 = pickle.load(open("2021Dict.p","rb"))
dict2022 = pickle.load(open("2022Dict.p","rb"))

#testing if the import worked
dict2020["Ravens"]

DT      90.3
QB      90.1
LT      89.4
TE      88.5
CB      83.9
RB      78.7
LB      74.4
S       76.9
RT      73.6
WR      72.8
EDGE    69.2
C       68.7
LG      63.8
RG      63.6
dtype: object

In [52]:
#Deshaun Watson is most likely going to be suspended for an 1+ season
#so I will assume the Browns will start Jacoby Brissett for 2022
#the Browns also traded away Baker Mayfield to the Panthers
dict2022["Browns"]["QB"] = 76.3
dict2022["Browns"]
dict2022["Panthers"]["QB"] = 63.6

In [53]:
#import game results from trimmed csvs
games2020 = pd.read_csv("2020schedtrimmed.csv")
games2020

,For,Against,Differential
0,Kansas City Chiefs,Houston Texans,14
1,Seattle Seahawks,Atlanta Falcons,13
2,Buffalo Bills,New York Jets,10
3,Las Vegas Raiders,Carolina Panthers,4
4,Chicago Bears,Detroit Lions,4
...,...,...,...
507,Denver Broncos,Las Vegas Raiders,-1
508,Houston Texans,Tennessee Titans,-3
509,Kansas City Chiefs,Los Angeles Chargers,-17
510,San Francisco 49ers,Seattle Seahawks,-3


In [54]:
#shortening each team name
games2020["For"] = games2020["For"].apply(lambda x: x.split(" ")[-1])
games2020["Against"] = games2020["Against"].apply(lambda x: x.split(" ")[-1])
games2020

,For,Against,Differential
0,Chiefs,Texans,14
1,Seahawks,Falcons,13
2,Bills,Jets,10
3,Raiders,Panthers,4
4,Bears,Lions,4
...,...,...,...
507,Broncos,Raiders,-1
508,Texans,Titans,-3
509,Chiefs,Chargers,-17
510,49ers,Seahawks,-3


In [55]:
#simplfy rows
games2020["gamename"] = games2020["For"] + " " + games2020["Against"] + " " + "2020"
games2020 = games2020.drop(columns = ["For", "Against"])
games2020

,Differential,gamename
0,14,Chiefs Texans 2020
1,13,Seahawks Falcons 2020
2,10,Bills Jets 2020
3,4,Raiders Panthers 2020
4,4,Bears Lions 2020
...,...,...
507,-1,Broncos Raiders 2020
508,-3,Texans Titans 2020
509,-17,Chiefs Chargers 2020
510,-3,49ers Seahawks 2020


In [56]:
#for each game, add predictors representing the strength of players from the "For" team and the "Against" team
positions = ["QB", "WR", "TE", "RB", "DT", "LT", "CB", "LB", "S", "RT", "EDGE", "C", "LG", "RG"]
for pos in positions:
        games2020[pos + " " + "FOR"] = games2020["gamename"].apply(lambda x: dict2020[x.split(" ")[0]].loc[pos])
for pos in positions:
        games2020[pos + " " + "AGAINST"] = games2020["gamename"].apply(lambda x: dict2020[x.split(" ")[1]].loc[pos])
games2020 = games2020.set_index("gamename")
games2020

,Differential,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Chiefs Texans 2020,14,90.2,84.5,86.1,65.7,87.6,71.2,71.9,50.4,75.4,...,59.8,75.1,64.5,68.6,76.7,59.4,87.0,67.3,59.1,53.4
Seahawks Falcons 2020,13,92.0,81.9,79.6,77.9,68.4,74.1,87.6,76.4,73.4,...,90.1,79.7,56.8,76.6,72.0,53.0,72.0,72.1,87.7,47.0
Bills Jets 2020,10,64.1,78.3,60.2,69.3,65.1,73.1,74.3,65.6,77.7,...,81.3,74.4,79.0,73.9,87.9,60.6,73.1,72.0,56.1,65.6
Raiders Panthers 2020,4,79.9,75.5,83.2,87.1,73.1,64.9,67.4,79.0,69.3,...,83.7,79.3,63.8,65.9,76.4,76.2,54.7,63.4,58.6,63.6
Bears Lions 2020,4,76.6,80.8,58.0,66.6,91.7,58.6,62.5,61.9,66.8,...,85.9,75.5,82.8,72.3,74.4,74.4,83.0,74.9,64.9,78.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Broncos Raiders 2020,-1,57.6,83.1,52.0,71.3,78.4,76.1,81.3,88.6,90.8,...,73.1,64.9,67.4,79.0,69.3,69.1,65.5,71.0,74.0,61.8
Texans Titans 2020,-3,82.4,75.2,56.6,75.9,59.8,75.1,64.5,68.6,76.7,...,72.4,80.2,79.9,68.8,78.3,71.7,65.4,79.7,77.3,46.6
Chiefs Chargers 2020,-17,90.2,84.5,86.1,65.7,87.6,71.2,71.9,50.4,75.4,...,69.5,63.3,84.2,66.6,82.4,77.1,89.6,58.2,51.7,63.9


In [57]:
#Repeat this process for the 2021 and 2022 seasons
games2021 = pd.read_csv("2021schedtrimmed.csv")

games2022 = pd.read_csv("2022schedtrimmed.csv")

games2021["For"] = games2021["For"].apply(lambda x: x.split(" ")[-1])
games2021["Against"] = games2021["Against"].apply(lambda x: x.split(" ")[-1])

games2022["For"] = games2022["For"].apply(lambda x: x.split(" ")[-1])
games2022["Against"] = games2022["Against"].apply(lambda x: x.split(" ")[-1])

games2021["gamename"] = games2021["For"] + " " + games2021["Against"] + " " + "2021"
games2021 = games2021.drop(columns = ["For", "Against"])

games2022["gamename"] = games2022["For"] + " " + games2022["Against"] + " " + "2022"
games2022 = games2022.drop(columns = ["For", "Against"])

for pos in positions:
        games2021[pos + " " + "FOR"] = games2021["gamename"].apply(lambda x: dict2021[x.split(" ")[0]].loc[pos])
for pos in positions:
        games2021[pos + " " + "AGAINST"] = games2021["gamename"].apply(lambda x: dict2021[x.split(" ")[1]].loc[pos])
games2021 = games2021.set_index("gamename")

for pos in positions:
        games2022[pos + " " + "FOR"] = games2022["gamename"].apply(lambda x: dict2022[x.split(" ")[0]].loc[pos])
for pos in positions:
        games2022[pos + " " + "AGAINST"] = games2022["gamename"].apply(lambda x: dict2022[x.split(" ")[1]].loc[pos])
games2022 = games2022.set_index("gamename")

In [58]:
games2021

,Differential,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Buccaneers Cowboys 2021,2,93.3,82.2,70.9,65.5,89.9,72.8,76.4,78.8,70.0,...,76.5,76.6,62.7,89.8,72.0,86.4,88.6,53.5,70.8,91.3
Eagles Falcons 2021,26,56.2,71.2,79.5,68.2,74.1,70.3,62.9,60.2,70.8,...,80.2,75.5,64.7,68.7,65.3,64.3,57.8,47.0,49.2,77.1
Steelers Bills 2021,7,69.0,75.5,57.5,70.7,89.5,57.4,68.9,59.7,76.4,...,58.6,80.6,68.5,55.6,71.3,79.2,75.6,63.3,53.8,64.8
Panthers Jets 2021,5,58.4,79.1,61.7,86.5,68.0,61.6,70.4,83.3,59.0,...,81.4,74.4,73.1,73.6,82.9,79.9,76.3,62.2,66.8,63.0
Bengals Vikings 2021,3,75.1,83.1,61.4,76.7,69.6,70.1,67.0,54.7,90.1,...,75.1,71.9,70.9,82.6,74.3,78.0,89.0,61.4,66.2,54.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Giants Commanders 2021,-15,78.4,81.6,66.5,72.2,79.8,62.4,79.9,75.9,70.0,...,78.4,74.9,71.4,64.8,84.1,74.9,87.1,76.4,65.9,86.3
Ravens Steelers 2021,-3,79.3,74.6,76.5,85.5,74.7,79.9,76.4,71.5,69.8,...,89.5,57.4,68.9,59.7,76.4,78.6,91.6,52.4,66.2,34.8
Cardinals Seahawks 2021,-8,82.8,87.1,67.7,69.4,85.4,88.3,74.5,59.9,75.3,...,79.8,87.8,80.2,85.1,63.1,71.4,68.6,59.8,71.2,63.7


In [59]:
games2022

,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,RT FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Bills Rams 2022,90.9,81.5,63.6,69.0,71.8,79.4,70.3,50.4,83.8,59.3,...,93.5,77.7,84.3,71.8,74.3,83.4,73.8,74.8,68.4,48.2
Saints Falcons 2022,74.2,79.6,62.7,67.6,81.9,69.8,77.3,79.9,82.9,84.1,...,67.6,71.3,82.6,44.5,58.8,62.8,65.3,77.1,49.2,83.7
Browns Panthers 2022,76.3,73.0,70.9,81.7,41.6,66.1,76.9,76.5,68.7,78.8,...,65.4,71.6,61.3,72.7,71.4,77.5,63.8,73.3,61.6,69.6
49ers Bears 2022,59.9,90.2,90.7,76.8,76.0,96.6,71.2,80.1,74.8,69.8,...,57.9,61.4,64.2,63.7,59.0,61.4,70.4,57.5,66.2,51.8
Steelers Bengals 2022,63.5,73.2,72.6,69.6,90.8,57.5,75.8,37.7,58.0,62.2,...,79.5,74.5,75.5,61.5,72.1,80.2,73.4,72.4,54.2,71.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Browns Steelers 2022,76.3,73.0,70.9,81.7,41.6,66.1,76.9,76.5,68.7,78.8,...,90.8,57.5,75.8,37.7,58.0,62.2,89.6,69.7,64.5,71.0
Chiefs Raiders 2022,80.4,70.0,85.0,65.6,83.4,75.4,79.1,72.5,70.1,67.2,...,61.3,85.0,79.4,63.9,71.0,45.5,91.7,64.8,56.7,50.6
Rams Seahawks 2022,86.1,93.0,69.1,75.7,93.5,77.7,84.3,71.8,74.3,83.4,...,75.4,66.6,70.2,59.7,71.2,54.0,64.3,70.3,57.1,63.6


In [60]:
#categorizing data into forecast and modeling
forecast = games2022
modeling = pd.concat([games2020, games2021])
modeling

,Differential,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Chiefs Texans 2020,14,90.2,84.5,86.1,65.7,87.6,71.2,71.9,50.4,75.4,...,59.8,75.1,64.5,68.6,76.7,59.4,87.0,67.3,59.1,53.4
Seahawks Falcons 2020,13,92.0,81.9,79.6,77.9,68.4,74.1,87.6,76.4,73.4,...,90.1,79.7,56.8,76.6,72.0,53.0,72.0,72.1,87.7,47.0
Bills Jets 2020,10,64.1,78.3,60.2,69.3,65.1,73.1,74.3,65.6,77.7,...,81.3,74.4,79.0,73.9,87.9,60.6,73.1,72.0,56.1,65.6
Raiders Panthers 2020,4,79.9,75.5,83.2,87.1,73.1,64.9,67.4,79.0,69.3,...,83.7,79.3,63.8,65.9,76.4,76.2,54.7,63.4,58.6,63.6
Bears Lions 2020,4,76.6,80.8,58.0,66.6,91.7,58.6,62.5,61.9,66.8,...,85.9,75.5,82.8,72.3,74.4,74.4,83.0,74.9,64.9,78.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Giants Commanders 2021,-15,78.4,81.6,66.5,72.2,79.8,62.4,79.9,75.9,70.0,...,78.4,74.9,71.4,64.8,84.1,74.9,87.1,76.4,65.9,86.3
Ravens Steelers 2021,-3,79.3,74.6,76.5,85.5,74.7,79.9,76.4,71.5,69.8,...,89.5,57.4,68.9,59.7,76.4,78.6,91.6,52.4,66.2,34.8
Cardinals Seahawks 2021,-8,82.8,87.1,67.7,69.4,85.4,88.3,74.5,59.9,75.3,...,79.8,87.8,80.2,85.1,63.1,71.4,68.6,59.8,71.2,63.7


In [61]:
forecast.to_pickle("forecast.pkl")
modeling.to_pickle("modeling.pkl")